##  <center>Cones representing a 3d vector field</center>

A cone plot  represents a 3D vector field, $(x,y,z)\mapsto(u(x,y,z), v(x,y,z), w(x,y,z))$ defined on a subset in the 3D space.
The  cone trace is defined by the 1d arrays (vectors)  of x, y, z-coordinates for the vector positions, and the u, v, w-as the coordinates of the vector field. The direction of the vector field at a point is illustrated by a geometric cone, colored according to the vector norm at that point. 
    

### Cones defined by the coordinates read from a dataframe

In [ ]:
using HTTP
using CSV
using DataFrames
using PlotlyJS
include("src/plotlyju.jl");
include("src/PlotlyColorSchemes.jl");

In [ ]:
url = "https://raw.githubusercontent.com/plotly/datasets/master/vortex.csv" 
df = CSV.File(HTTP.get(url).body)|> DataFrame;

In [ ]:
trace1 =  cone(x=df[!,"x"],
              y=df[!, "y"],
              z=df[!, "z"],
              u=df[!, "u"],
              v=df[!, "v"],
              w=df[!, "w"],
              sizemode="absolute",
              sizeref=40,
              colorscale=plotly_cs(ice), 
              colorbar_len=0.7,
              anchor="tip"
           );

layout1 = Layout(
               width=650,
               height=650,
               margin=attr(t=10, r=10, b=10, l=10),
               scene_camera_eye=attr(x=1.45, y=1.45, z=0.7),
               scene_aspectratio=attr(x=1, y=1, z=0.8)
)
pl1 = plot(trace1, layout1, style=plotlyju)

![vortex](images/vortex.png)

### The tangent vector field along the coordinate lines of a parameterized hyperboloid

The one-sheet hyperboloid, $\displaystyle\frac{x^2}{a^2}-\frac{y^2}{b^2}+\frac{z^2}{a^2}-1=0$, is parameterized
by: 

$$\begin{array}{lll}x(u,v)&=& a u \sin(v)\sqrt{1+u^2}\\
                    y(u,v)&=& b u\\
                    z(u,v)&=&a u \cos(v)\sqrt{1+u^2}\end{array}$$

The tangent vector field along a line of constant v has the components:
    
$$\begin{array}{lll}x'_u&=&\displaystyle\frac{a u \sin(v)}{\sqrt{1+u^2}}\\
               y'_u&=& b \\
               z'_u&=&\displaystyle\frac{a u \cos(v)}{\sqrt{1+u^2}}\end{array}$$

Parameterization functions for the hyperboloid corresponding to a=1, b=2, as well for the tangent vector field:

In [ ]:
f, g, h = (u, v) -> sin(v)*sqrt(1+u^2),  (u, v) -> 2u, (u, v) -> cos(v)*sqrt(1+u^2)
tanx, tany, tanz = (u, v) -> sin(v)*u/sqrt(1+u^2), (u, v)->2, (u, v) -> cos(v)* u/sqrt(1+u^2)

n = 15
ul = -5:5/(n-2):0
vl = 0:2π/(n-1):2π
#flattened meshgrid
u, v = vec([ui for ui in ul, vi in vl]), vec([vi for ui in ul, vi in vl])

trace2 = cone(x=f.(u, v), 
              y=g.(u,v),  
              z=h.(u, v), 
              u=tanx.(u, v), 
              v=tany.(u, v), 
              w=tanz.(u ,v),
              sizemode="absolute",
              sizeref=2.0,
              showscale=false,
              colorscale= [[0, "#bd1540"],
                           [1, "#bd1540"]], 
              colorbar_len=0.65, 
              anchor="tail",
              opacity=0.8
           )
layout2 = Layout(title="Cone plot of a tangent vector field", 
                 width=700,
                 height=550,
                 scene_camera_eye=attr(x=1.6, y=1.6, z=0.6));
pl2 = plot(trace2, layout2,  style=plotlyju)  

![hyp-tan-cones](images/tan-cones.png)

### Cones along a trajectory of the Roessler system 

$$\begin{array}{lll} \dot{x}&=&-y-z\\
                     \dot{y}&=&x+ay,\quad\quad a=0.2,\: b=0.2,\: c=5.7\\
    \dot{z}&=&b+z(x-c)\end{array}$$

In [ ]:
using DifferentialEquations
using StaticArrays

In [ ]:
function Roessler!(du, u, p, t)   
    a, b, c = p
    du[1] = -u[2]-u[3]
    du[2] = u[1]+a*u[2]
    du[3] = b+ u[3]*(u[1]-c)
end

In [ ]:
u0 = [0.6, 1, 0 ] #initial condition
p = (0.2, 0.2, 5.7)
tspan = (0.0,  75)
prob = ODEProblem(Roessler!, u0, tspan, p)
sol = solve(prob);
t = 0:0.035:75; 

s  = sol.(t)
s = Vector{SVector{3, Float64}}(s)
smatr = reshape(reinterpret(Float64, s), (3, length(s)))';

traj = scatter3d(x=smatr[:, 1],
                 y=smatr[:, 2],
                 z=smatr[:, 3], 
                 mode="lines",
                 line_width=2, 
                 line_color="rgb(102,135,231)")

Place cones at the points of index in I:

In [ ]:
I = collect(1:8:length(s))
cpos = smatr[ I, :]  
x = cpos[:, 1]
y = cpos[:, 2]
z = cpos[:, 3]
u = @. -y-z
v = @. x+0.2*y
w = @. 0.2+z*(x-5.7);

cone_tr = cone(x=x,
             y=y,
             z=z,
             u=u,
             v=v,
             w=w,
             sizemode="absolute",
             sizeref=30,  
             colorscale=pl_ice, 
             colorbar_len=0.7,
             anchor="center"
             )
pl3 = plot([traj, cone_tr], Layout(width=600, height=600,
                                   scene_xaxis_visible=false, 
                                   scene_yaxis_visible=false,
                                   scene_zaxis_visible=false,
                                   scene_camera_eye=attr(x=-1.5, y=-1.5,  z=0.5),
                                   scene_aspectmode="data"), style=plotlyju)


![Roessler](images/Roessler.png)